In [1]:
import pandas as pd
from time import sleep
import os
from selenium import webdriver # for interacting with website
from selenium.webdriver.common.by import By
import re

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
def transcript2df(transcript):
    pattern = r'^(([0-9]{2}|[0-9]{1}):([0-9]{2}|[0-9]{1})|([0-9]{2}|[0-9]{1})):[0-9]{2}$'
    transcript_timestamps,transcript_dialog,prev = [],[],None
    for item in transcript:
        if re.match(pattern, item):
            transcript_timestamps.append(item)
            if prev == None: 
                prev = item
            else: 
                transcript_dialog.append(prev)
                prev = item
        else:
            if re.match(pattern, prev):
                prev = item
            else:
                prev = prev + ' ' + item
    transcript_dialog.append(item)
    df = pd.DataFrame({'timestamp':transcript_timestamps, 
                   'text':transcript_dialog})    
    return df

In [13]:
def get_youtube_transcript(video_ids):
    frames = []
    driver = webdriver.Firefox()
    for video_id in video_ids:
        try:
            driver.get(f"https://www.youtube.com/watch?v={video_id}")
            sleep(4)
            driver.find_element(By.XPATH,'//*[@id="button-shape"]/button').click()
            sleep(2)
            driver.find_element(By.XPATH,'//*[@id="items"]/ytd-menu-service-item-renderer[2]/tp-yt-paper-item/yt-formatted-string').click()
            sleep(4)
            transcript_element = driver.find_element(By.XPATH,"//*[@id='body']/ytd-transcript-segment-list-renderer")
            transcript = transcript_element.text
            transcript = transcript.split('\n')
            transcript.pop(0)
            frames.append(transcript2df(transcript).assign(video_id=video_id))
        except:
            print(video_id)
    driver.quit()
    return pd.concat(frames,ignore_index=True)

In [6]:
df = pd.read_parquet('../../data/scrap_oso_trava/playlist_videos.parquet')
df

,id,title,description,published_at,playlist_id,channel_id,photo_url
0,fL9FNpuGK_8,"SUSO ZAMORA | Perder a TODA tu FAMILIA, lo que...",Suso Zamora es un inversionista peruano fundad...,2023-08-28T03:19:53Z,PL5EDqo-XIoDCg5zTnqp0s6mWMPyex7ukN,UCWVspRnU4QdsR4Av4tDWWRw,https://i.ytimg.com/vi/fL9FNpuGK_8/maxresdefau...
1,uTS8L1kvjmY,"SEBASTIÁN KREIS | Lecciones de NICO SZEKASY, D...",Sebastian Kreis es Co-fundador y CEO de Xepeli...,2023-08-07T13:39:30Z,PL5EDqo-XIoDCg5zTnqp0s6mWMPyex7ukN,UCWVspRnU4QdsR4Av4tDWWRw,https://i.ytimg.com/vi/uTS8L1kvjmY/maxresdefau...
2,027NOppIrVQ,"FERNANDO LANDEROS | El Infierno en México, SOU...",VUELVETE ANGEL DE FREEDOM EN ESTE LINK: 👉 http...,2023-07-24T12:22:59Z,PL5EDqo-XIoDCg5zTnqp0s6mWMPyex7ukN,UCWVspRnU4QdsR4Av4tDWWRw,https://i.ytimg.com/vi/027NOppIrVQ/maxresdefau...
3,2u6FeKJ-biw,LUIS CERVANTES I La INVERSION PERFECTA #236,Luis Cervantes en LinkedIn como Luis Cervantes...,2023-07-17T13:23:16Z,PL5EDqo-XIoDCg5zTnqp0s6mWMPyex7ukN,UCWVspRnU4QdsR4Av4tDWWRw,https://i.ytimg.com/vi/2u6FeKJ-biw/maxresdefau...
4,OGwMD9NMQMA,"UPDATE SHOW con Javier Morodo I Shark Tank, MA...",Javier es un inversionista mexicano fundador d...,2023-07-10T13:24:55Z,PL5EDqo-XIoDCg5zTnqp0s6mWMPyex7ukN,UCWVspRnU4QdsR4Av4tDWWRw,https://i.ytimg.com/vi/OGwMD9NMQMA/maxresdefau...
...,...,...,...,...,...,...,...
991,qQCp1jqQmd8,SERGIO ROSENGAUS | La Revolución de la INTELIG...,Sergio Rosengaus (Twitter: @sergiorosengaus) e...,2023-03-06T14:11:46Z,PL5EDqo-XIoDAHEMIB8WkvLgRY7eUJ3MNJ,UCWVspRnU4QdsR4Av4tDWWRw,https://i.ytimg.com/vi/qQCp1jqQmd8/maxresdefau...
992,7y2y9lTs5r0,"PEDRO DE GARAY | El Mundo de las Inversiones, ...","Pedro De Garay es CoCEO de GBM, empresa que bu...",2023-04-10T12:59:51Z,PL5EDqo-XIoDAHEMIB8WkvLgRY7eUJ3MNJ,UCWVspRnU4QdsR4Av4tDWWRw,None
993,2u6FeKJ-biw,LUIS CERVANTES I La INVERSION PERFECTA #236,Luis Cervantes en LinkedIn como Luis Cervantes...,2023-07-17T13:23:16Z,PL5EDqo-XIoDAHEMIB8WkvLgRY7eUJ3MNJ,UCWVspRnU4QdsR4Av4tDWWRw,https://i.ytimg.com/vi/2u6FeKJ-biw/maxresdefau...
994,eGwzC6GAEvE,Convierte tu miedo en tu mayor ventaja,Con este simple truco llamado reapreciación de...,2019-12-27T17:17:51Z,PL5EDqo-XIoDCaAUZJ4tOFPX8OTq04QQmq,UCWVspRnU4QdsR4Av4tDWWRw,https://i.ytimg.com/vi/eGwzC6GAEvE/maxresdefau...


In [10]:
df_aux = df[df.id.isin(df.id.unique()[20:25])].copy().drop_duplicates(subset=['id'])
df_aux

,id,title,description,published_at,playlist_id,channel_id,photo_url
28,fQYtqIELCVE,"#089. Alberto Lati - Adaptabilidad, Prejuicios...","Entrevista de Oso Trava con Alberto Lati, el r...",2022-11-22T04:17:57Z,PL5EDqo-XIoDD67TGNUdw2K94jFUqf7b5Z,UCWVspRnU4QdsR4Av4tDWWRw,https://i.ytimg.com/vi/fQYtqIELCVE/maxresdefau...
29,eZUdfDDatpI,"LUIS ALVAREZ | El Mundo del WHISKY, Llevar el ...",Luis Álvarez @luisalvarezironman es un empresa...,2022-12-19T13:25:59Z,PL5EDqo-XIoDD67TGNUdw2K94jFUqf7b5Z,UCWVspRnU4QdsR4Av4tDWWRw,https://i.ytimg.com/vi/eZUdfDDatpI/maxresdefau...
32,7y2y9lTs5r0,"PEDRO DE GARAY | El Mundo de las Inversiones, ...","Pedro De Garay es CoCEO de GBM, empresa que bu...",2023-04-10T12:59:51Z,PL5EDqo-XIoDBtGuOa0495kdwH7mnlrQOs,UCWVspRnU4QdsR4Av4tDWWRw,None
33,nYE_PI1g2G4,"Javier Martínez Morodo | Update Show: Ego, Cam...","Mi invitado de hoy es Javier Martínez Morodo, ...",2022-07-05T21:28:36Z,PL5EDqo-XIoDB_aexKjjbd8IIjkB4JtwBx,UCWVspRnU4QdsR4Av4tDWWRw,https://i.ytimg.com/vi/nYE_PI1g2G4/maxresdefau...
34,Zi298e7ohUw,"Amaury Vergara | Continuar su Legado, Lidiar c...",Amaury Vergara es presidente y director genera...,2022-07-11T13:15:49Z,PL5EDqo-XIoDB_aexKjjbd8IIjkB4JtwBx,UCWVspRnU4QdsR4Av4tDWWRw,https://i.ytimg.com/vi/Zi298e7ohUw/maxresdefau...


In [28]:
print(df_aux.description.iloc[0])

Entrevista de Oso Trava con Alberto Lati, el reconocido conductor y comentarista deportivo mexicano.

“Si no te adaptas no vives.” – Alberto Lati @latitudlati

Alberto es periodista y escritor. En sus más de 25 años de carrera ha cubierto historias en más de cien países, habiendo residido en ocho de ellos.

Durante su carrera, que incluye la cobertura 5 Copas del Mundo y 4 Juegos Olímpicos de verano, ha podido entrevistar, hablando 12 idiomas diferentes, a las mayores glorias del deporte, a ocho Premios Nobel de la Paz, escritores, músicos y numerosos jefes de Estado

Su primer libro, LATITUDES. CRÓNICA, VIAJE Y BALÓN, se convirtió en Best Seller y desde entonces ha publicado 4 libros más cerrando este 2020 con 100 dioses de Olimpo al que añadió, recientemente, 20 pelotazos de esperanza en tiempos de crisis, escrito en plena cuarentena.

Desde 2015, es colaborador del Alto Comisionado de Naciones Unidas para los Refugiados. 

Hoy Alberto y yo hablamos de prejuicios, de adaptabilidad y 

In [14]:
result = get_youtube_transcript(df_aux.id)
result

nYE_PI1g2G4
Zi298e7ohUw


,timestamp,text,video_id
0,0:00,el punto central de lo que tiene una filosofía...,fQYtqIELCVE
1,0:05,aprendido algo que es adaptación si tú no te a...,fQYtqIELCVE
2,0:11,llegué a japón pensando que japón ciudad a par...,fQYtqIELCVE
3,0:17,de crecimiento [Música],fQYtqIELCVE
4,0:31,hola y bienvenidos a un nuevo episodio de crac...,fQYtqIELCVE
...,...,...,...
2774,1:26:46,quieres recibir todos los bienes un correo muy...,7y2y9lTs5r0
2775,1:26:53,frases artículos cosas que me recomiendan mis ...,7y2y9lTs5r0
2776,1:27:00,creo que pueden ayudarte a tener una vida más ...,7y2y9lTs5r0
2777,1:27:07,de cracks es totalmente gratis y puedes recibi...,7y2y9lTs5r0


In [26]:
result.video_id.value_counts()

video_id
eZUdfDDatpI    1071
fQYtqIELCVE     894
7y2y9lTs5r0     814
Name: count, dtype: int64

In [24]:
result.text.loc[1820]

'vivir un departamento y apliqué la dineroterapia dije Ay cómo sí quiero'

In [ ]:
result.to_parquet('../scripts/data/scrap_oso_trava/videos_s',)